In [1]:
import os
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
google_api_key=os.environ['GEMINI_API_KEY']

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=google_api_key)

#testing
response=llm.invoke("what is 2+3")
print(response.content)

c:\Users\panka\OneDrive\Desktop\Euron\GenAi\9.Level_1_Application_Development\Level-1-Apllication-Development\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2 + 3 = 5


In [3]:
from langchain_community.utilities import SQLDatabase
path_to_db="Data/street_tree_db.sqlite"

db=SQLDatabase.from_uri(f"sqlite:///{path_to_db}")


In [4]:
db

In [5]:
##Our Approach:
    # we know we cannot directly query the database we have to convert or NL to Query
    #then we get an ans and then convert that ans to NL

#Step 1. Converting our Natural Language Question to Query

In [12]:
from langchain.chains import create_sql_query_chain

chain=create_sql_query_chain(llm,db)

res=chain.invoke({"question":"How many species of tress are in San Francisco ?"})

In [20]:
res[res.rfind("\n")+11:]

'SELECT COUNT(DISTINCT "qSpecies") FROM street_trees'

In [21]:
db.run(res[res.rfind("\n")+11:])

'[(148,)]'

In [22]:
#inspecting the chain for its prompts
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#step 2: Executing the SQL Query

In [29]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query=create_sql_query_chain(llm,db)
execute_query=QuerySQLDataBaseTool(db=db)

chain1=write_query | execute_query

res=chain1.invoke({"question":"How many species of tress are in San Francisco ?"})

In [31]:
print(res)
"""
yaha error is liye de rha hai kyuki humara LLM sahi answer nahi kar rha hai..
"""

Error: (sqlite3.OperationalError) near "Question": syntax error
[SQL: Question: How many species of tress are in San Francisco ?
SQLQuery: SELECT COUNT(DISTINCT "qSpecies") FROM street_trees]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


#Step 3: Translate SQL response into Natural Language

In [33]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, 
    corresponding SQL query, and SQL result, 
    answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

write_query = create_sql_query_chain(llm, db)

execute_query = QuerySQLDataBaseTool(db=db)

#chain = write_query | execute_query

#chain.invoke({"question": "List the species of trees that are present in San Francisco"})

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

'The provided SQL query resulted in an error. The correct query to answer the question should be something like:\n\n```sql\nSELECT DISTINCT "qSpecies" \nFROM street_trees\nWHERE "PlantType" = \'Tree\' AND "qCaretaker" = \'DPW\' AND "qCity" = \'San Francisco\';\n```\n\nSince the query failed, I cannot provide a list of tree species present in San Francisco.  The error message indicates a syntax error near "Question:", suggesting that the SQL interpreter is trying to parse the text preceding the actual SQL query.  This extra text needs to be removed.'